## Data Analysis Document
Author : PengboHu

In [95]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

## Data clear


In [4]:
#read all files

import os
from os.path import join

AllFile = []
def main() :
    dest = "h:\data\lab-data-processing\data"
    for root, dirs, files in os.walk( dest ):
        for OneFileName in files :
            if OneFileName.find( '.txt' ) == -1 :
                continue
            OneFullFileName = join( root, OneFileName )
            AllFile.append(OneFullFileName)
        return AllFile

In [5]:
file = main()
len(file)

30

#### Detail information extract

In [6]:
class DataContentDecode:
    """Get the detail information
    
    This class help to decode the file and get the describe information
    and data information.
    
    Attributes:
        debug(int):             debug mode. 0 close and 1 open
        
        trailName(int):         the number of the trail 
        pelteirNumber(int):     the number of the peltier 0 - 8
        subPelterNumber(int):   the index of the sub peltier 1 or 2
        sex(int):               male 1   female 0
        used(int):              used 1    unused 0
        experimenter(int):      Hu 1   YZH 0
        protocol(int):          A 1   B 0
        rawData(pd.DataFrame):  data source
    
    """
    def __init__(self,fileName):
        self.fileName = fileName      #the .txt file name
        self.debug = 0
    
    def fileDecode(self):             #get the detail information
        f = open(self.fileName,"r")
        d = f.readlines()
        headNumber = int(d[0][-5:-3])-2
        
        ''' get detail describe of data'''
        for i in range(headNumber-2):
            if(d[i].find("Trial name")!=-1):
                self.trailName = int(re.findall(r"[0-9]+",d[i])[0])
                if(self.debug):
                    print("trainName:",int(re.findall(r"[0-9]+",d[i])[0]))
            if(d[i].find("Arena name")!=-1):
                numFind = re.findall(r"[0-9]+",d[i])
                self.peltierNumber = int(numFind[0])
                self.subPeltierNumber = int(numFind[-1])
                if(self.debug):
                    print("peltierNumber:",peltierNumber,"subPeltierNumber:",subPeltierNumber)
            if(d[i].find("Sex")!=-1):
                if(d[i][7:-3]=="male"):
                    self.sex = 1
                else:
                    self.sex = 0
                if(self.debug):
                    print("sex:",sex)
            if(d[i].find("Used")!=-1):
                if(d[i][8:-3]=="Used"):
                    self.used = 1
                else:
                    self.used = 0               
                if(self.debug):
                    print("used:",used)
            if(d[i].find("Experimenter")!=-1):
                if(d[i][16:-3]=="Hu"):
                    self.experimenter = 1
                else:
                    self.experimenter = 0
                if(self.debug):
                    print("experimenter:",experimenter)
            if(d[i].find("Protocol")!=-1):
                if(d[i][12:-3]=="A"):
                    self.protocol = 1
                else:
                    self.protocol = 0
                if(self.debug):
                    print("protocol:",protocol)
        
        ''' get raw data '''
        head = [j for i,j in enumerate(list(d[int(d[0][-5:-3])-2].split('"'))) if i%2 !=0 ]
        if(self.debug):
            print(head)
        dataFrame = {i:[] for i in head}

        for i in d[headNumber+2:]:
            for index,data in zip(dataFrame,i.split(";")[:-1]):
                if data=='"-"':
                    data=np.nan
                dataFrame[index].append(data)
        self.rawData = pd.DataFrame(dataFrame)
        if(self.debug):
            print(self.rawData.head())
        
        
        f.close()

#### Extract all file

In [7]:
class GetAllFile:
    """Get the all file 
    
    This class help to decode all the file and get their detail information
    
    Attributes:
        dataContentDecodeList:  list of DataContentDecode Oject
    
    """
    def __init__(self,fileList):
        self.fileList = fileList
        self.dataContentDecodeList = []
    
    def getDataContentDecodeList(self):
        for i in self.fileList:
            t = DataContentDecode(i)
            t.fileDecode()
            self.dataContentDecodeList.append(t)
    

In [8]:
t  = GetAllFile(file)
t.getDataContentDecodeList()

In [9]:
bb = t.dataContentDecodeList

In [10]:
j  = 0
for i in bb:
    if i.sex == 1:
        j+=1
    print(i.trailName,i.peltierNumber,i.sex,i.experimenter)
print( "number of 5 :",j )

5 5 1 1
5 5 1 1
5 6 1 1
5 6 1 1
5 7 1 1
5 7 1 1
5 8 1 1
5 8 1 1
6 5 1 1
6 5 1 1
6 6 1 1
6 6 1 1
6 7 1 1
6 7 1 1
6 8 1 1
6 8 1 1
7 5 1 1
7 5 1 1
7 6 1 1
7 6 1 1
7 7 1 1
7 7 1 1
7 8 1 1
7 8 1 1
8 5 1 1
8 5 1 1
8 6 1 1
8 6 1 1
8 8 1 1
8 8 1 1
number of 5 : 30


#### Data filter

In [11]:
class DataAnalysis:
    """Data analysis 
    
    This class help to analyze data
    
    Attributes:
        dataContentDecodeList:  list of DataContentDecode Oject
    
    """
    def __init__(self):
            t = GetAllFile(file)
            t.getDataContentDecodeList()
            self.extractedFile = t.dataContentDecodeList
    
   
    
    
    
    
    #--------------------------- Filter module start ---------------------#
        
    
    """
    trailName = -1   means all trail
    trailName: list
    """
    def dataFilterTrailName(self,trailName,rawData):
        result = []
        if trailName == -1:
            return rawData
        else:
            for i in trailName:
                for j in rawData:
                    if(j.trailName == i):
                        result.append(j)
        return result
    
    
    '''
    Experimenter = -1 means all trail
    1 : Hu          
    0 : ZhengHong
    '''
    def dataFilterExperimenter(self,experimenter,rawData):
        result = []
        if experimenter == -1:
            return rawData
        else:
            for j in rawData:
                if(j.experimenter == experimenter):
                    result.append(j)
        return result
            
    
    '''
    1  :  Used
    0  :  unUsed
    '''
    def dataFilterUsed(self,used,rawData):
        result = []
        for j in rawData:
            if(j.used == used):
                result.append(j)
        return result
    
    
    '''
    peltierNumber(list):    [0-8]
    '''
    def dataFilterPeltierNumber(self,peltierNumber,rawData):
        result = []
        if peltierNumber == -1:
            return rawData
        else:
            for i in peltierNumber:
                for j in rawData:
                    if(j.peltierNumber == i):
                        result.append(j)
        return result
    
    
    '''
    sex(int):  male 1,  female 0,   All -1
    '''
    def dataFilterSex(self,sex,rawData):
        result = []
        if sex == -1:
            return rawData
        else:
            for j in rawData:
                if(j.sex == sex):
                    result.append(j)
        return result
        
    
    '''
    protocol(int):  A  1,  B  0,  All  -1
    '''
    def dataFilterProtocol(self,protocol,rawData):
        result = []
        if(protocol == -1):
            return rawData
        
        for j in rawData:
            if(j.protocol == protocol):
                result.append(j)
        return result
    
    
    #--------------------------- Filter module end  ---------------------#
    '''
    Get all trailName
    '''
    def trailNameObtainInData(self):
        trailName = []
        for i in self.extractedFile:
            trailName.append(i.trailName)
        
        return set(trailName)
    
    
    '''
    choose data from the following parapeters
    '''
    def chooseData(self,trailName,peltierNumber,used,sex,experimenter,protocol):
        result = self.dataFilterTrailName(trailName,self.extractedFile)
        result = self.dataFilterExperimenter(experimenter,result)
        result = self.dataFilterUsed(used,result)
        result = self.dataFilterProtocol(protocol,result)
        result = self.dataFilterPeltierNumber(peltierNumber,result)
        result = self.dataFilterSex(protocol,result)
        return result
    
    
    
    
    '''
    input you filter information
    '''
    def dataFilter(self):
        rawData = self.extractedFile
    
        print("Please input filter information")
        print("Detail parameters")
        print("   trailName:     input [1,2,3]  or -1(all trail)")
        print("                  trailName choosed form",self.trailNameObtainInData())
        print("   peltierNumber: input [1,2,3] or -1(all peltierNumber)")
        print("   used:          used 1, unused 0")
        print("   sex:           male 1, female 0, All -1")
        print("   experimenter:  Hu 1,  YZH 0, All -1")
        print("   protocol:      A 1, B 0, All -1")
        print("")
        
        print("Please input you choose")
        
        trailName = eval(input("TrailName:"))
        while(True):
            if(type(trailName)==type([]) or trailName == -1):
                break;
            else:
                print("input wrong,please input again")
                trailName = input("TrainName:")
        
        
        
                
                
        
        peltierNumber = eval(input("PeltierNumber:"))
        while(True):
            if(type(peltierNumber) == type([]) or peltierNumber == -1):
                break;
            else:
                print("input wrong,please input again")
                peltierNumber = eval(input("PeltierNumber:"))
                
        used = eval(input("Used:"))
        while(True):
            if(used== 0 or used==1):
                break;
            else:
                print("input wrong,please input again")
                used = eval(input("Used:"))
        
        
        sex = eval(input("Sex:"))
        while(True):
            if(sex== 0 or sex==1 or sex==-1 ):
                break;
            else:
                print("input wrong,please input again")
                sex = eval(input("Sex:"))
                
                
        experimenter = eval(input("Experimenter:"))
        while(True):
            if(experimenter== 0 or experimenter==1 or experimenter==-1 ):
                break;
            else:
                print("input wrong,please input again")
                experimenter = eval(input("Experimenter:"))
        
        
        
        
        protocol = eval(input("Protocol:"))
        while(True):
            if(protocol== 0 or protocol==1 or protocol==-1 ):
                break;
            else:
                print("input wrong,please input again")
                protocol = eval(input("Protocol:"))
    
        self.filterData = self.chooseData(trailName,peltierNumber,used,sex,experimenter,protocol)
        print(" ")
        print("Number of data",len(self.filterData))
                
    

In [12]:
#         trailName(int):         the number of the trail 
#         pelterNumber(int):      the number of the pelter 0 - 8
#         subPelterNumber(int):   the index of the sub pelter 1 or 2
#         sex(int):               male 1   female 0
#         used(int):              used 1    unused 0
#         experimenter(int):      Hu 1   YZH: 0
#         protocol(int):          A 1   B 0
#         rawData(pd.DataFrame):  data source

In [13]:

# t = DataAnalysis()
# a = t.dataFilterTrailName(-1,t.extractedFile)

t = DataAnalysis()
a = t.dataFilterProtocol(-1,t.extractedFile)

In [14]:
result = t.dataFilter()

Please input filter information
Detail parameters
   trailName:     input [1,2,3]  or -1(all trail)
                  trailName choosed form {8, 5, 6, 7}
   peltierNumber: input [1,2,3] or -1(all peltierNumber)
   used:          used 1, unused 0
   sex:           male 1, female 0, All -1
   experimenter:  Hu 1,  YZH 0, All -1
   protocol:      A 1, B 0, All -1

Please input you choose
TrailName:-1
PeltierNumber:-1
Used:1
Sex:-1
Experimenter:-1
Protocol:-1
 
Number of data 30


In [15]:
data = t.filterData
data


#### Group data to master and yorked

In [16]:
## Group data with master and yorked
for i in data:
    print(i.trailName,i.peltierNumber,i.subPeltierNumber,i.protocol)
    

5 5 1 0
5 5 2 0
5 6 1 0
5 6 2 0
5 7 1 0
5 7 2 0
5 8 1 0
5 8 2 0
6 5 1 0
6 5 2 0
6 6 1 0
6 6 2 0
6 7 1 0
6 7 2 0
6 8 1 0
6 8 2 0
7 5 1 0
7 5 2 0
7 6 1 0
7 6 2 0
7 7 1 0
7 7 2 0
7 8 1 0
7 8 2 0
8 5 1 0
8 5 2 0
8 6 1 0
8 6 2 0
8 8 1 0
8 8 2 0


In [73]:
## Group data with master and yorked
class GroupData:
    '''
    Group data by their trailName and peltierNumber
    
    Attribute:
        data:                     data after filter
        master,yorked:            data divide by the property of master and yorked
        AllTrailName:             set of trail name
        dataDivideByTrailName:    dic of data by trail name
        
    '''
    
    def __init__(self,data):
        self.data = data
        self.master = []
        self.yorked = []
    
    
    '''
    get the set of trailName.
    '''
    def __getTrailName(self):
        self.AllTrailName = []
        for i in data:
            self.AllTrailName.append(i.trailName)
        self.AllTrailName = set(self.AllTrailName)
    
    
    '''
    group data by trailName and produce a dic with key of trailName
    '''
    def __groupData(self):
        self.__getTrailName()
        self.dataDivideByTrailName = {}
        for i in self.AllTrailName:
            self.dataDivideByTrailName[i]=[]
            for dataItem in self.data:
                if dataItem.trailName == i:
                    self.dataDivideByTrailName[i].append(dataItem)
    
    '''
    sort the data to make the data who has same trailName and peltierNumber 
    get together
    '''
    def __groupToMasterAndYorked(self):
        self.TempData = []
        for key in self.dataDivideByTrailName.keys():
            dicLength = len(self.dataDivideByTrailName[key])
            dicContent = self.dataDivideByTrailName[key]
            for i in range(dicLength-1):
                for j in range(i+1,dicLength):
                    if dicContent[i].peltierNumber == dicContent[j].peltierNumber:
                        self.TempData.append(dicContent[i])
                        self.TempData.append(dicContent[j])
            
       
                
    '''
    finial group function to get the master group and yorked group
    '''            
    def GetAllGroup(self):
        self.__groupData()
        self.__groupToMasterAndYorked()
        for i in range(len(self.TempData)):
            if i%2 == 0:
                pass
            else:
                if self.TempData[i].protocol == 1:
                    if self.TempData[i].subPeltierNumber == 1:
                        self.master.append(self.TempData[i])
                        self.yorked.append(self.TempData[i-1])
                    else:
                        self.yorked.append(self.TempData[i])
                        self.master.append(self.TempData[i-1])
                else:
                    if self.TempData[i].subPeltierNumber == 1:
                        self.yorked.append(self.TempData[i])
                        self.master.append(self.TempData[i-1])
                    else:
                        self.master.append(self.TempData[i])
                        self.yorked.append(self.TempData[i-1])
                    
    
        
        

In [74]:
gd = GroupData(data)
gd.GetAllGroup()

In [83]:
for i,j in zip(gd.master,gd.yorked):
    print("master",i.trailName,i.peltierNumber,i.subPeltierNumber)
    print("yorked",j.trailName,j.peltierNumber,j.subPeltierNumber)

master 8 5 2
yorked 8 5 1
master 8 6 2
yorked 8 6 1
master 8 8 2
yorked 8 8 1
master 5 5 2
yorked 5 5 1
master 5 6 2
yorked 5 6 1
master 5 7 2
yorked 5 7 1
master 5 8 2
yorked 5 8 1
master 6 5 2
yorked 6 5 1
master 6 6 2
yorked 6 6 1
master 6 7 2
yorked 6 7 1
master 6 8 2
yorked 6 8 1
master 7 5 2
yorked 7 5 1
master 7 6 2
yorked 7 6 1
master 7 7 2
yorked 7 7 1
master 7 8 2
yorked 7 8 1


# Analysis and visualization

In [110]:
class AnalysisData:
    '''
    Get the detail information from the filtered data
    
    '''
    
    def __init__(self,master,yorked):
        self.master = master 
        self.yorked = yorked
    
    
    
    
    
    
    
    

In [111]:
ad = AnalysisData(gd.master,gd.yorked)

In [115]:
gd.master[0].rawData.head()

,Area,Areachange,Elongation,Movement,Pretest,Recording time,Test,Training,Trial time,X center,Y center
0,3.00191,0.529749,0.384809,NaN,1,0.000,0,0,0.000,-21.08,14.5703
1,NaN,NaN,NaN,NaN,1,0.040,0,0,0.040,NaN,NaN
2,NaN,NaN,NaN,NaN,1,0.080,0,0,0.080,NaN,NaN
3,NaN,NaN,NaN,NaN,1,0.120,0,0,0.120,NaN,NaN
4,3.17849,0.353166,0.515148,NaN,1,0.160,0,0,0.160,-20.9376,14.587


In [116]:
len(gd.master[0].rawData)

10501